In [1]:
import Base: iterate,max, exp, sin, cos, tan, +, ^, -, *, /, sqrt, convert, promote_rule, zero,isless
using BenchmarkTools
import Test: @test, @testset

In [2]:
2^2

4

# Define dual number

In [3]:
struct Dual{T <: Number} <: Number
    x::T
    dx::T
end
Dual(n::Integer, d::Float64) = Dual(promote(n, d)...)
Dual(n::Float64, d::Integer) = Dual(promote(n, d)...)

Dual

In [4]:
function convert(::Type{Dual{T}}, x::T) where {T}
   Dual(x, zero(x)) 
end
function convert(::Type{Dual{T}}, x::Dual{S}) where {S, T}
    Dual(T(x.x), T(x.dx))
end
# This is needed according to an error before
function convert(::Type{Dual{T}}, x::T) where {T <: Number}
    Dual(x, zero(x))
end
function convert(::Type{Dual{T}}, x::S) where {T, S <: Number}
    x_as_T = convert(T, x)
    Dual(x_as_T, zero(x_as_T))
end

convert (generic function with 187 methods)

In [5]:
function zero(x::Dual{T}) where T
    Dual(zero(x.x), zero(x.dx))
end

zero (generic function with 23 methods)

In [6]:
function promote_rule(::Type{Dual{T}}, ::Type{Dual{S}}) where {T,S}
    Dual{promote_type(T,S)}
end
function promote_rule(::Type{Dual{T}}, ::Type{S}) where {T, S <: Number}
    Dual{promote_type(T,S)}
end
function promote_rule(::Type{T}, ::Type{Dual{S}}) where {T <: Number, S}
    Dual{promote_type(T,S)}
end
function promote_rule(::Type{S}, ::Type{Dual{T}}) where {S <: AbstractIrrational, T}
    Dual{promote_type(S,T)}
end

promote_rule (generic function with 126 methods)

In [7]:
function extract_derivative(xdx::Dual)
    return xdx.dx
end
function extract_derivative(xs::Array)
    [extract_derivative(x) for x in xs]
end
function extract_derivative(xs::Tuple)
    convert(Tuple, [extract_derivative(x) for x in xs])
end

extract_derivative (generic function with 3 methods)

# Differential operator

In [8]:
function derivativeСalculation(f, value)
    function df(x)
        xdx = Dual(x, one(x))
        result = f(xdx)
        return extract_derivative(result)
    end
    df(value)
end
function derivativeСalculation(f,i::Integer,values)
    function df(valuesS)
        xarr = [(j != i ? x : Dual(x, one(x))) for (j,x) in enumerate(valuesS)]
        result = f(xarr...)
        return extract_derivative(result)
    end
    df(values)
end
function derivativeСalculation(f,values)
    function df(xs...)
        xarr = [(j != i ? x : Dual(x, one(x))) for (j,x) in enumerate(xs)]
        result = f(xarr...)
        return extract_derivative(result)
    end
    df(values)
end

derivativeСalculation (generic function with 2 methods)

In [9]:
function +(x::Dual, y::Dual)
    Dual(x.x+y.x, x.dx+y.dx)
end
function -(x::Dual, y::Dual)
    Dual(x.x-y.x, x.dx-y.dx)
end
function -(x::Dual)
    Dual(-x.x, -x.dx)
end
function *(x::Dual, y::Dual)
#      println("mnożenie")
     Dual(x.x*y.x, x.x*y.dx + x.dx*y.x)
end
function /(x::Dual, y::Dual)
    Dual(x.x/y.x, x.dx/y.x - x.x*y.dx/(y.x*y.x))
end
function /(x::Array, y::Number)
    return [i / y for i in x]
end

function ^(a::Dual, x::Dual)
#     println("Daszek")
    Dual(a.x^x.x, a.dx * x.x * a.x ^ (x.x - 1) + x.dx * a.x ^ x.x * log(a.x)) 
end

function ^(a::Dual, x::Integer)
#     println("Daszek")
    Dual(a.x^x, a.dx * x * a.x ^ (x - 1)) 
end

^ (generic function with 68 methods)

In [10]:
function sqrt(x::Dual)
    Dual(sqrt(x.x), x.dx/(2*sqrt(x.x)))
end

sqrt (generic function with 21 methods)

In [11]:
function exp(x::Dual)
    return Dual(exp(x.x), exp(x.x)*x.dx)
end
function exp(xs::Array)
    print(xs)
    return [exp(x) for x in xs]
end
function sin(x::Dual)
    return Dual(sin(x.x), cos(x.x)*x.dx)
end
function cos(x::Dual)
    return Dual(cos(x.x), -sin(x.x)*x.dx)
end

function tan(x::Dual)
    return Dual(tan(x.x),(1/cos(x.x)^2)*x.dx)
end
isless(x::Dual, y::Dual) = x.x < y.x;

In [12]:
function max(x::Dual)
    return Dual(max(0,x.x), x.x < 0 ? 0 : 1 * x.dx)
end

max (generic function with 14 methods)

In [13]:
function softmax(vector::Array)
    e = exp(vector)
    return e / sum(e)
#     z = [Dual(2.0, 1.0), Dual(21.0, 1.0)]
#     s = zero(vector[1])
#     for x in z
#        s = s + x 
#     end    
#     return e / s
end

softmax (generic function with 1 method)

In [14]:
function iterate(iter::Vector{Dual}, state=1)
#     println(iter, "Test22232323")
    if state > length(iter)
        return nothing
    end
    return (iter[state],state+1)
end

iterate (generic function with 214 methods)

In [15]:
J = function jacobian(f, args::Vector{T}) where {T <:Number}
    jacobian_columns = Matrix{T}[]
    
    for i=1:length(args)
        x = Dual{T}[]
        for j=1:length(args)
            seed = (i == j)
            push!(x, seed ?
                Dual(args[j], one(args[j])) :
                Dual(args[j],zero(args[j])) )
        end
        temp  = [f(x)...]
        column = extract_derivative.([f(x)...])
        push!(jacobian_columns, column[:,:])
    end
    hcat(jacobian_columns...)
end

jacobian (generic function with 1 method)

In [16]:
f(x::Vector) = softmax(x)
J(softmax, [1., 2.])

Dual{Float64}[Dual{Float64}(1.0, 1.0), Dual{Float64}(2.0, 0.0)]Dual{Float64}[Dual{Float64}(1.0, 1.0), Dual{Float64}(2.0, 0.0)]Dual{Float64}[Dual{Float64}(1.0, 0.0), Dual{Float64}(2.0, 1.0)]Dual{Float64}[Dual{Float64}(1.0, 0.0), Dual{Float64}(2.0, 1.0)]

2×2 Array{Float64,2}:
  0.196612  -0.196612
 -0.196612   0.196612

In [17]:
f(x::Vector) = [x[1], 5/x[3], 4x[2]^2-2x[3], x[3]*sin(x[1]), exp(x[1])/sum(x)]
J(f, [1, 2., 3])

5×3 Array{Float64,2}:
 1.0        0.0         0.0
 0.0        0.0        -0.555556
 0.0       16.0        -2.0
 1.62091    0.0         0.841471
 0.377539  -0.0755078  -0.0755078

In [18]:
function linear(x)
    return x^2
end


linear (generic function with 1 method)

In [19]:
derivativeСalculation(linear,1,[12])

24

In [20]:
softmax([1,2,3])

[1, 2, 3]

3-element Array{Float64,1}:
 0.09003057317038046
 0.24472847105479767
 0.6652409557748219

In [22]:
function relu(x)
    return max(0,x)
end

relu (generic function with 1 method)

In [27]:
function rosenbrock(x::Vector)
    value = zero(x[1])
    for i=2:length(x)
        value += (1-x[i-1])^2 + 100*(x[i] - x[i-1]^2)^2
    end
    value
end


rosenbrock (generic function with 1 method)

In [28]:
# d = derivativeСalculation(rosenbrock,[1.0,2.0,3.0])
@benchmark J(rosenbrock, [rand(1000,1)...])

BenchmarkTools.Trial: 
  memory estimate:  32.30 MiB
  allocs estimate:  21021
  --------------
  minimum time:     61.908 ms (0.00% GC)
  median time:      64.369 ms (2.13% GC)
  mean time:        64.577 ms (1.65% GC)
  maximum time:     72.270 ms (2.55% GC)
  --------------
  samples:          78
  evals/sample:     1

In [29]:
[rand(3,1)...]

3-element Array{Float64,1}:
 0.8370579275291241
 0.27071793418662704
 0.5353063942871525

In [30]:
s = rand(3,1)
print(s[1][1])

0.4314945157017869

In [31]:
J(rosenbrock, [1., 2., 3, 4, 5,6 ,7])

1×7 Array{Float64,2}:
 -400.0  1002.0  5804.0  16606.0  35808.0  65810.0  -5800.0

In [32]:
rosenbrock([1,2])

100

In [33]:
function rosenbrock(x)
    value = zero(x[1])
#     value = (1-a)^2 + 100*(x - a^2)^2
    for i=2:length(x)
        value += (1-x[i-1])^2 + 100*(x[i] - x[i-1]^2)^3
    end
    value
end

rosenbrock (generic function with 2 methods)

In [34]:
derivativeСalculation(rosenbrock,1,[1,2,3])

LoadError: [91mMethodError: no method matching rosenbrock(::Dual{Int64}, ::Int64, ::Int64)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  rosenbrock(::Any) at In[33]:1[39m

In [35]:
tan(2)

-2.185039863261519

In [36]:
f(x::Vector) = [x[1], 5/x[3], 4x[2]^2-2x[3], x[3]*sin(x[1]), exp(x[1])/sum(x), x[1]^2 - x[2]^3 - 88]
z = @benchmark J(f,[rand(3,1)...]) 

BenchmarkTools.Trial: 
  memory estimate:  5.20 KiB
  allocs estimate:  77
  --------------
  minimum time:     3.502 μs (0.00% GC)
  median time:      3.682 μs (0.00% GC)
  mean time:        4.507 μs (8.16% GC)
  maximum time:     433.528 μs (98.74% GC)
  --------------
  samples:          10000
  evals/sample:     8

In [37]:
J(f, [2.9,2.2,1546.23])

6×3 Array{Float64,2}:
     1.0          0.0          0.0
     0.0          0.0         -2.09133e-6
     0.0         17.6         -2.0
 -1501.32         0.0          0.239249
     0.0117077   -7.55172e-6  -7.55172e-6
     5.8        -14.52         0.0

In [41]:
println(z.params)


BenchmarkTools.Parameters(5.0, 10000, 8, 0.0, true, false, 0.05, 0.01)


5.203125

In [47]:
dump(z)
rand(3)

BenchmarkTools.Trial
  params: BenchmarkTools.Parameters
    seconds: Float64 5.0
    samples: Int64 10000
    evals: Int64 8
    overhead: Float64 0.0
    gctrial: Bool true
    gcsample: Bool false
    time_tolerance: Float64 0.05
    memory_tolerance: Float64 0.01
  times: Array{Float64}((10000,)) [3501.75, 3510.875, 3514.75, 3518.5, 3526.5, 3528.75, 3529.0, 3529.25, 3532.625, 3532.875  …  14832.5, 399484.0, 400900.125, 401069.625, 402952.875, 413437.375, 414294.375, 423066.125, 433005.25, 433527.875]
  gctimes: Array{Float64}((10000,)) [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 394007.5, 396021.625, 396344.625, 398242.125, 407772.625, 409225.0, 418216.5, 427972.75, 428045.25]
  memory: Int64 5328
  allocs: Int64 77


3-element Array{Float64,1}:
 0.45342592920396085
 0.9567364943983228
 0.15937741341114897

In [48]:
struct TestFunc
    fun
    numberOfSubfunctions::Integer
end

In [81]:
function styblinskiTang(x)
    value = zero(x[1])
    for i=1:length(x)
        value += x[i]^4 - 16 * x[i]^2 + 5 * x[i]
    end
    value / 2
end

function rosenbrock(x)
    value = zero(x[1])
    for i=2:length(x)
        value += (1-x[i-1])^2 + 100*(x[i] - x[i-1]^2)^2
    end
    value
end

function KarolikJeleniewiczFunction(x)
    f1 = 222.83*(x[2]^8+x[3]^-3)^2 + sin(x[5]^18) - cos(x[1])/exp(x[3])
    f2 = x[1] - 5/x[2] + 4*x[3]^2-2*x[4] / (x[3]*sin(x[5])^2 + 1)
    f3 = sqrt(exp(x[2])^2) + x[3]^x[2] - x[4]
    [f1,f2,f3]
end

function viennet(x)
    f1 = 0.5*(x[1]^2+x[2]^2) + sin(x[1]^2+ x[2]^2)
    f2 = (x[1]-2*x[2]+4)^2/8 + (x[1]-x[2]+1)^2/27 + 15
    f3 = 1/(x[1]^2+x[2]^2+1) - 1.1* exp(-(x[1]^2+x[2]^2))
    [f1,f2,f3]
end


viennet (generic function with 1 method)

In [82]:
J(KarolikJeleniewiczFunction, [2.9,2.2,2.2,34456.5,1.8])

3×5 Array{Float64,2}:
 0.0265096   4.88099e8  -31325.0      0.0           1.77884e5
 1.0         1.03306      6878.31    -0.647997  -7042.76
 0.0        13.493           5.6667  -1.0           0.0

In [89]:
D = Dict("viennet" => [viennet, 2], "KarolikJeleniewiczFunction" => [KarolikJeleniewiczFunction, 5])

Dict{String,Array{Any,1}} with 2 entries:
  "viennet"                    => Any[viennet, 2]
  "KarolikJeleniewiczFunction" => Any[KarolikJeleniewiczFunction, 5]

In [126]:
    for key in keys(D)
           informationAboutFunction = get(D, key, Nothing)
            println(informationAboutFunction[2])
    end

2
5


In [180]:
function makeTestWektorFunction(definition)
    benchamarkResults = []
    for key in keys(definition)
        print(key)
       informationAboutFunction = get(definition, key, Nothing)
        println(informationAboutFunction[2])
       push!(benchamarkResults, d(viennet,[rand(is[2],1)...])))
#         print(J(informationAboutFunction[1],[rand(informationAboutFunction[2],1)...]))
    end
    benchamarkResults
end


LoadError: [91msyntax: extra token ")" after end of expression[39m

In [174]:
x = makeTestWektorFunction(D)


viennet2


LoadError: [91mUndefVarError: informationAboutFunction not defined[39m